In [1]:
# Load the SQL extension first (only shows once)
%load_ext sql

# Import required libraries
import os
from dotenv import load_dotenv
import urllib

# Load variables from your .env file (with UTF-8 encoding)
load_dotenv(encoding='utf-8-sig')

# Fetch variables
server = os.getenv("SERVER")
database = os.getenv("DATABASE")
driver = os.getenv("DRIVER")

# Build the connection string
params = urllib.parse.quote_plus(
    f"DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
)
connection_string = f"mssql+pyodbc:///?odbc_connect={params}"

%sql {connection_string}
%config SqlMagic.displaycon = False  # Hide connection info
print("Secure connection established successfully!")

Secure connection established successfully!


Topic: Subquery in SELECT (Derived Column)

Question
Display each employee’s first name, last name, and vacation hours, along with the company-wide average vacation hours.


In [2]:
%%sql
SELECT TOP 3  /* Displaying first 3 records to limit output */
    PEP.FirstName,
    PEP.LastName,
    HRE.VacationHours,
    (SELECT AVG(VacationHours) FROM HumanResources.Employee) AS CompanyAverageVacationHours
FROM HumanResources.Employee AS HRE
LEFT JOIN Person.Person AS PEP
    ON HRE.BusinessEntityID = PEP.BusinessEntityID;

Done.


FirstName,LastName,VacationHours,CompanyAverageVacationHours
James,Hamilton,64,50
Peter,Krebs,43,50
Jo,Brown,80,50


Topic: Subquery in FROM (Derived Table)

Question:
Find the average pay rate per job title, then return only those job titles with an average pay rate greater than 50.

In [3]:
%%sql
SELECT
	JobTitle,
	AveragePayRate
FROM 
	(SELECT
		HRE.[JobTitle],
		AVG(EPH.[Rate]) AS AveragePayRate
	FROM
		HumanResources.Employee HRE
	LEFT JOIN HumanResources.EmployeePayHistory EPH
	ON HRE.BusinessEntityID = EPH.BusinessEntityID
	GROUP BY HRE.[JobTitle]) AS JobTitleSummary
WHERE 
	AveragePayRate > 50;

Done.


JobTitle,AveragePayRate
Chief Executive Officer,125.5000
Information Services Manager,50.4808
Vice President of Engineering,63.4615
Vice President of Production,84.1346
Vice President of Sales,72.1154


Topic: Comparing to a Department (Subquery in WHERE)

Question:
List employees whose vacation hours are greater than the average vacation hours of the Tool Design department


In [4]:
%%sql
SELECT TOP 3  /* Displaying first 3 records to limit output */
	HRE.[BusinessEntityID],
    PEP.FirstName,
    PEP.LastName,
    HRE.VacationHours
FROM HumanResources.Employee AS HRE
JOIN Person.Person AS PEP
    ON HRE.BusinessEntityID = PEP.BusinessEntityID
WHERE 	HRE.[VacationHours] > 
	(SELECT 
		AVG(E.[VacationHours])
	FROM HumanResources.Employee E
	LEFT JOIN HumanResources.EmployeeDepartmentHistory DH
	ON E.BusinessEntityID = DH.BusinessEntityID
	LEFT JOIN HumanResources.Department D
	ON DH.DepartmentID = D.DepartmentID
	WHERE D.[Name] = 'Tool Design') 

Done.


BusinessEntityID,FirstName,LastName,VacationHours
1,Ken,Sánchez,99
4,Rob,Walters,48
7,Dylan,Miller,61


Topic: Comparing to a Department (Subquery in WHERE)

Question: List employees (BusinessEntityID, First Name, Last Name, Vacation Hours) whose vacation hours are greater than the average vacation hours of the entire company

In [5]:
%%sql
SELECT  TOP 3 /* Displaying first 3 records to limit output */
	HRE.[BusinessEntityID],
	PEP.[FirstName],
	PEP.[LastName],
	HRE.[VacationHours]
FROM
	HumanResources.Employee HRE
LEFT JOIN
	Person.Person PEP
ON HRE.BusinessEntityID = PEP.BusinessEntityID
WHERE
	HRE.[VacationHours] >
		(SELECT AVG([VacationHours]) FROM HumanResources.Employee)

Done.


BusinessEntityID,FirstName,LastName,VacationHours
1,Ken,Sánchez,99
7,Dylan,Miller,61
8,Diane,Margheim,62


Topic: Correlated Subquery – Per-Row Comparison

Question: List all products whose ListPrice is greater than the average ListPrice of their respective ProductSubcategory.

Tables: Production.Product, Production.ProductSubcategory

In [6]:
%%sql
SELECT TOP 3  /* Displaying first 3 records to limit output */
    PRP.ProductID,
    PRP.Name AS ProductName,
    PRP.ListPrice,
    PSC.Name AS SubcategoryName,
    (   SELECT 
            AVG(PRP2.ListPrice)
        FROM 
            Production.Product AS PRP2
        WHERE 
            PRP2.ProductSubcategoryID = PRP.ProductSubcategoryID
    ) AS AvgListPriceInSubcategory
FROM Production.Product AS PRP
LEFT JOIN Production.ProductSubcategory AS PSC
    ON PRP.ProductSubcategoryID = PSC.ProductSubcategoryID
WHERE 
    PRP.ListPrice > (
        SELECT 
            AVG(PRP2.ListPrice)
        FROM 
            Production.Product AS PRP2
        WHERE 
            PRP2.ProductSubcategoryID = PRP.ProductSubcategoryID);

Done.


ProductID,ProductName,ListPrice,SubcategoryName,AvgListPriceInSubcategory
771,"Mountain-100 Silver, 38",3399.9900,Mountain Bikes,1683.3650
772,"Mountain-100 Silver, 42",3399.9900,Mountain Bikes,1683.3650
773,"Mountain-100 Silver, 44",3399.9900,Mountain Bikes,1683.3650


Topic: EXISTS – Check for Relationships

Question: List all vendors who have supplied at least one product.

Tables: Purchasing.Vendor, Purchasing.ProductVendor

In [7]:
%%sql
SELECT V.BusinessEntityID, V.Name
FROM Purchasing.Vendor AS V
WHERE EXISTS (
    SELECT 'Vendor Found'
    FROM Purchasing.ProductVendor AS PV
    WHERE PV.BusinessEntityID = V.BusinessEntityID
);

Done.


BusinessEntityID,Name
1492,Australia Bike Retailer
1494,Allenson Cycles
1496,Advanced Bicycles
1498,"Trikes, Inc."
1500,Morgan Bike Accessories
1504,Chicago Rent-All
1506,Greenwood Athletic Company
1508,"Compete Enterprises, Inc"
1510,International
1514,Training Systems


In [8]:
%%sql
--not using EXISTS 

SELECT TOP 3  /* Displaying first 3 records to limit output */
	PV.[BusinessEntityID],
	PV.[Name],
	COUNT(PPV.[ProductID]) AS ProductSold
FROM
	Purchasing.Vendor PV
LEFT JOIN
	Purchasing.ProductVendor PPV
ON PV.BusinessEntityID = PPV.BusinessEntityID
GROUP BY 
	PV.[BusinessEntityID],
	PV.[Name]
HAVING COUNT(PPV.[ProductID]) >= 1
ORDER BY COUNT(PPV.[ProductID]) DESC;

Done.


BusinessEntityID,Name,ProductSold
1656,Mountain Works,30
1540,Bergeron Off-Roads,24
1536,Cruger Bike Company,23


Topic: NOT EXISTS – Find Orphans

Question:Show all employees who have never been assigned to any department.

Tables: HumanResources.Employee, HumanResources.EmployeeDepartmentHistory

In [9]:
%%sql
SELECT v.BusinessEntityID,
       v.AccountNumber,
       v.Name
FROM Purchasing.Vendor AS v
WHERE NOT EXISTS (
    SELECT 1
    FROM Purchasing.ProductVendor AS pv
    WHERE pv.BusinessEntityID = v.BusinessEntityID
);

Done.


BusinessEntityID,AccountNumber,Name
1502,CYCLING0001,Cycling Master
1512,LIGHTSP0001,Light Speed
1524,RECREATI0001,Recreation Place
1528,IMAGEMA0001,Image Makers Bike Center
1532,KNOPFLER0001,Knopfler Cycles
1550,MERITBI0001,Merit Bikes
1552,SPORTSH0001,Sports House
1558,MARSH0001,Marsh
1564,ILLINOIS0001,Illinois Trek & Clothing
1596,ADATUM0001,A. Datum Corporation


In [10]:
%%sql
--not using NOT EXISTS

SELECT
	PV.[BusinessEntityID],
	PV.[Name],
	COUNT(PPV.[ProductID]) AS ProductSold
FROM
	Purchasing.Vendor PV
LEFT JOIN
	Purchasing.ProductVendor PPV
ON PV.BusinessEntityID = PPV.BusinessEntityID
GROUP BY 
	PV.[BusinessEntityID],
	PV.[Name]
HAVING COUNT(PPV.[ProductID]) = 0
ORDER BY COUNT(PPV.[ProductID]) DESC;

Done.


BusinessEntityID,Name,ProductSold
1502,Cycling Master,0
1512,Light Speed,0
1524,Recreation Place,0
1528,Image Makers Bike Center,0
1532,Knopfler Cycles,0
1550,Merit Bikes,0
1552,Sports House,0
1558,Marsh,0
1564,Illinois Trek & Clothing,0
1596,A. Datum Corporation,0


Topic: IN – Multi-Value Subquery

Question:
List all products that have been ordered in sales orders placed by customers located in the same territories as SalesPersons with a quota history.

Tables: Production.Product, Sales.SalesOrderDetail, Sales.SalesOrderHeader, Sales.SalesPersonQuotaHistory, Sales.SalesPerson

In [11]:
%%sql

SELECT 
    DISTINCT TOP 3  P.Name AS ProductName /* Displaying first 3 records to limit output */
FROM Production.Product P
JOIN Sales.SalesOrderDetail SOD
    ON P.ProductID = SOD.ProductID
JOIN Sales.SalesOrderHeader SOH
    ON SOD.SalesOrderID = SOH.SalesOrderID
WHERE SOH.TerritoryID IN (
    SELECT DISTINCT TerritoryID
    FROM Sales.SalesPersonQuotaHistory SPQ
    JOIN Sales.SalesPerson SP
        ON SPQ.BusinessEntityID = SP.BusinessEntityID
    WHERE SP.TerritoryID IS NOT NULL
);

Done.


ProductName
All-Purpose Bike Stand
AWC Logo Cap
Bike Wash - Dissolver


Topic: Subquery with JOIN – Complex Filtering

Question:
List all salespersons who have a quota higher than any other salesperson in their territory.

Tables: Sales.SalesPerson, Sales.SalesPersonQuotaHistory

In [12]:
%%sql
SELECT TOP 3  /* Displaying first 3 records to limit output */
	SP.[BusinessEntityID],
	SQH.SalesQuota Quota,
	SP.TerritoryID
FROM Sales.SalesPerson SP
JOIN Sales.SalesPersonQuotaHistory SQH
ON SP.BusinessEntityID = SQH.BusinessEntityID
WHERE SP.TerritoryID IS NOT NULL
AND SQH.SalesQuota > ANY
	(
	SELECT
		SQH2.SalesQuota Quota
	FROM Sales.SalesPerson SP2
	JOIN Sales.SalesPersonQuotaHistory SQH2
	ON SP2.BusinessEntityID = SQH2.BusinessEntityID
	WHERE SP2.TerritoryID = SP.TerritoryID
	AND SP2.BusinessEntityID != SP.BusinessEntityID
	)
ORDER BY SP.TerritoryID ASC;

Done.


BusinessEntityID,Quota,TerritoryID
280,165000.0000,1
280,469000.0000,1
280,264000.0000,1


Topic: Nested subqueries

Question:List products with the highest ListPrice in each ProductCategory.

Tables: Production.Product, Production.ProductSubcategory, Production.ProductCategory


In [13]:
%%sql

SELECT
	P.[Name] ProductName,
	P.[ListPrice],
	PC.[Name] CategoryName
FROM Production.Product P
JOIN Production.ProductSubcategory PSC
ON P.ProductSubcategoryID = PSC.ProductSubcategoryID
JOIN Production.ProductCategory PC
ON PSC.ProductCategoryID = PC.ProductCategoryID
WHERE P.[ListPrice] =
	(
		SELECT 
			MAX(P2.[ListPrice])
		FROM Production.Product P2
		JOIN Production.ProductSubcategory PSC2
		ON P2.ProductSubcategoryID = PSC2.ProductSubcategoryID
		JOIN Production.ProductCategory PC2
		ON PSC2.ProductCategoryID = PC2.ProductCategoryID
		WHERE PC.[Name] = PC2.Name
	)
ORDER BY PC.[Name]

Done.


ProductName,ListPrice,CategoryName
All-Purpose Bike Stand,159.0000,Accessories
"Road-150 Red, 62",3578.2700,Bikes
"Road-150 Red, 44",3578.2700,Bikes
"Road-150 Red, 48",3578.2700,Bikes
"Road-150 Red, 52",3578.2700,Bikes
"Road-150 Red, 56",3578.2700,Bikes
"Men's Bib-Shorts, S",89.9900,Clothing
"Men's Bib-Shorts, M",89.9900,Clothing
"Men's Bib-Shorts, L",89.9900,Clothing
"HL Road Frame - Black, 58",1431.5000,Components


Topic : Filtering by Top N Using Subquery

Question: Find the top 5 customers who have spent the most money on orders.

Tables: Sales.Customer, Sales.SalesOrderHeader

In [14]:
%%sql

SELECT
	TOP 5 Customers,
	TotalSpent
FROM
	(SELECT 
		S.[CustomerID] AS Customers,
		COUNT(SOH.[SalesOrderID]) AS TotalOrders,
		SUM(SOH.[TotalDue]) AS TotalSpent
	FROM Sales.Customer S
	JOIN Sales.SalesOrderHeader SOH
	ON S.CustomerID = SOH.CustomerID
	GROUP BY S.CustomerID) AS CustomerOrders
ORDER BY TotalSpent DESC

Done.


Customers,TotalSpent
29818,989184.0820
29715,961675.8596
29722,954021.9235
30117,919801.8188
29614,901346.8560


Topic: Subquery in CASE – Conditional Logic

Question: Display all customers with a column showing “Domestic” if their country is US, and “International” otherwise.

Tables: Sales.Customer, Person.Address, Person.StateProvince, Person.CountryRegion

In [16]:
%%sql

SELECT
	DISTINCT TOP 3 SC.[CustomerID], /* Displaying first 3 records to limit output */
	CASE	
		WHEN CR.[Name] = 'United States'
		THEN 'Domestic'
		ELSE 'International'
	END AS CustomerType
FROM Sales.Customer SC
LEFT JOIN Sales.SalesOrderHeader SOH
ON SC.CustomerID = SOH.CustomerID
LEFT JOIN Person.Address PA
ON SOH.BillToAddressID = PA.AddressID
LEFT JOIN Person.StateProvince SP
ON PA.StateProvinceID = SP.StateProvinceID
LEFT JOIN Person.CountryRegion CR
ON SP.CountryRegionCode = CR.CountryRegionCode;

Done.


CustomerID,CustomerType
1,International
2,International
7,International


Topic: Using Subquery to Filter by Count

Question:Show all products that appear in more than 5 transactions.

Tables: Production.Product, Production.TransactionHistory

In [17]:
%%sql

SELECT
	Products
FROM
	(
		SELECT
			[ProductID] Products,
			COUNT([SalesOrderID]) NumberofTransactions
		FROM [Sales].[SalesOrderDetail]
		GROUP BY [ProductID]
		HAVING COUNT([SalesOrderID]) > 1
	)AS ProductCount

Done.


Products
707
708
709
710
711
712
713
714
715
716
